In [135]:
    '''
We need to scrape:
    For each entity
        Pokemon name
        Stats
        Moves
        item
        Status
        Condition
    For the game:
        weather
        Entry hazards
        Terrain
        What pokemon we have out
    REMEMBER WHEN CONSTRUCTING INPUT WE HAVE TO KEEP TRACK OF WHICH PLAYER IS MOVING AND PUT THEIR TENSOR FIRST
'''

'\nWe need to scrape:\nFor each entity\n    Pokemon name\n    Stats\n    Moves\n    item\n    Status\n    Condition\nFor the game:\n    weather\n    Entry hazards\n    Terrain\n    What pokemon we have out\nREMEMBER WHEN CONSTRUCTING INPUT WE HAVE TO KEEP TRACK OF WHICH PLAYER IS MOVING AND PUT THEIR TENSOR FIRST\n'

In [140]:
import json
import pprint
%run PrintFunctions.ipynb
%run Databases.ipynb

In [148]:
class ScrapingFunctions:
    def splitTurnOutput(outputList):
        '''
        Function which splits output of turn into 3 parts in dict, p1, p2 and update
        '''
        outputDict = {}
        if 'p1' in outputList:
            p1index = outputList.index('p1')
            outputDict['p1'] = outputList[p1index + 1]
        else:
            outputDict['p1'] = ''
        if 'p2' in outputList:
            p2index = outputList.index('p2')
            outputDict['p2'] = outputList[p2index + 1]
        else:
            outputDict['p2'] = ''
        if 'update' in outputList:
            updateIndex = outputList.index('update')
            updateLines = [outputList[i] for i in range(updateIndex + 1, len(outputList))]
            outputDict['update'] = updateLines
        else:
            outputDict['update'] = []
        return outputDict
    def getPkmnLines(playerString):
        '''
        Turns string sent to terminal of player request into json
        '''
        jsonString = playerString[9:]
        print(jsonString)
        jsonPkmn = json.loads(jsonString)
        #pprint.pprint(jsonPkmn)
        return jsonPkmn
    def generateJson(outputList):
        '''
        Processes the output of a turn from the pipe into a readable format which we can use to
        generate an input for our neural net
        p1: JSON
        p2: JSON
        update: array
        '''
        outputDict = ScrapingFunctions.splitTurnOutput(outputList)
        if len(outputDict['update']) != 0:
            for i in outputDict['update']:
                print(i)
        if outputDict['p1'] != '':
            p1Dict = ScrapingFunctions.getPkmnLines(outputDict['p1'])
        else:
            assert(False)
            p1Dict = {}
        if outputDict['p2'] != '':
            p2Dict = ScrapingFunctions.getPkmnLines(outputDict['p2'])
        else:
            assert(False)
            p2Dict = {}
        return outputDict['update'], p1Dict, p2Dict
    def getPlayerDict(jsonDict):
        '''
        Turns json into nested dictionary with all inputs for neural net
        '''
        pkmnNames = {}
        pkmnHealth = {}
        pkmnStats = {}
        pkmnItems = {}
        pkmnMoves = {}
        pkmnStatus = {}
        playerDict = {}
        mainDictList = jsonDict['side']['pokemon']
        for pkmnDict in mainDictList:
            if 'p1' in pkmnDict['ident'] or 'p2' in pkmnDict['ident']:
                print("*" * 70)
                print(pkmnDict['ident'])
                name = pkmnDict['ident'][4:]
            else:
                name = pkmnDict['ident']
            if pkmnDict['active']:
                playerDict['active'] = name
            pkmnNames[name] = name
            statDict = pkmnDict['stats']
            statList = ['atk', 'def', 'spa', 'spd', 'spe']
            indivPkmnStatDict = {}
            for stat in statList:
                indivPkmnStatDict[stat] = statDict[stat]
            pkmnStats[name] = indivPkmnStatDict
            pkmnItems[name] = pkmnDict['item']
            pkmnMoves[name] = pkmnDict['moves']
            condition = pkmnDict['condition']
            if ' ' in condition:
                pkmnStatus[name] = condition.split(' ')[1]
            else:
                pkmnStatus[name] = ''
            conditionComponents = condition.split('/')
            curhealth = conditionComponents[0]
            try:
                totalhealth = conditionComponents[1]
            except IndexError:
                totalhealth = conditionComponents[0]
            if " " in totalhealth:
                totalhealth = totalhealth.split(" ")[0]
            if " " in curhealth:
                curhealth = curhealth.split(" ")[0]
            try:
                pkmnHealth[name] = float(curhealth)/float(totalhealth)
            except ZeroDivisionError:
                pkmnHealth[name] = 0.0
        if 'wait' in jsonDict:
            playerDict['wait'] = True
        if 'forceSwitch' in jsonDict:
            playerDict['forceSwitch'] = True
        try:
            playerDict['current'] = jsonDict['active'][0]['moves']
            if 'maybeTrapped' in jsonDict['active'][0] or 'trapped' in jsonDict['active'][0]:
                playerDict['maybeTrapped'] = True   
        except KeyError:
            pass
        playerDict['names'] = pkmnNames
        playerDict['health'] = pkmnHealth
        playerDict['stats'] = pkmnStats
        playerDict['items'] = pkmnItems
        playerDict['moves'] = pkmnMoves
        playerDict['status'] = pkmnStatus
        return playerDict
    def processCondtion(line, gameEffectDict):
        '''
        Edits condition dictionarys by checking line for changes in condition and editting if
        necessary
        '''
        if "-start|" in line:
            if 'p1' in line:
                for i in Databases.conditions['Name'].tolist():
                    if i in Databases.cleanVal(line):
                        p1Condition = i
                        gameEffectDict['conditions'] = [p1Condition, gameEffectDict['conditions'][1]]
            else:
                for i in Databases.conditions['Name'].tolist():
                    if i in Databases.cleanVal(line):
                        p2Condition = i
                        gameEffectDict['conditions'] = [gameEffectDict['conditions'][0], p2Condition]
        if "-end|" in line:
            if 'p1' in line:
                p1Condition = ''
                gameEffectDict['conditions'] = [p1Condition, gameEffectDict['conditions'][1]]
            else:
                p2Condition = ''
                gameEffectDict['conditions'] = [gameEffectDict['conditions'][0], p2Condition]
        return gameEffectDict
    def checkLineForWeather(line, gameEffectDict):
        if "weather" in line:
            if 'none' in line:
                gameEffectDict['weather'] = ''
            else:
                for i in Databases.weather['Name'].tolist():
                    if i in Databases.cleanVal(line):
                        print(f"SETTING WEATHER TO {i}")
                        gameEffectDict['weather'] = i
        return gameEffectDict
    def checkLineForField(line, gameEffectDict):
        if "-fieldstart" in line:
            for i in Databases.terrain['Name'].tolist():
                if i in Databases.cleanVal(line):
                    gameEffectDict['terrain'] = i
        if "-fieldend" in line:
            gameEffectDict['terrain'] = ''
        return gameEffectDict
    def checkSideEffects(line, gameEffectDict):
        if "-sidestart" in line:
            for i in Databases.sideEffects['Name'].tolist():
                if i in Databases.cleanVal(line):
                    if 'p1' in line:
                        gameEffectDict['p1sideeffect'].append(i)
                    else:
                        gameEffectDict['p2sideeffect'].append(i)
        if "-sideend" in line:
            for i in Databases.sideEffects['Name'].tolist():
                if i in line:
                    if 'p1' in Databases.cleanVal(line):
                        gameEffectDict['p1sideeffect'].remove(i)
                    else:
                        gameEffectDict['p2sideeffect'].remove(i)
        return gameEffectDict
    def processUpdates(updates, gameEffectDict):
        '''
        uses the ouput of generateJson to put our dictionaries in easier to read format
        '''
        for line in updates:
            gameEffectDict = ScrapingFunctions.checkLineForWeather(line, gameEffectDict)
            gameEffectDict = ScrapingFunctions.checkLineForField(line, gameEffectDict)
            gameEffectDict = ScrapingFunctions.checkSideEffects(line, gameEffectDict)
            gameEffectDict = ScrapingFunctions.processCondtion(line, gameEffectDict) 
        return gameEffectDict
    def getDicts(output, lastUpdates):
        updates, p1Json, p2Json = ScrapingFunctions.generateJson(output)
        if p1Json:
            p1Dict = ScrapingFunctions.getPlayerDict(p1Json)
        if p2Json:
            p2Dict = ScrapingFunctions.getPlayerDict(p2Json)
        #assumes json is generated when volatile status changes
        gameEffectDict = ScrapingFunctions.processUpdates(updates, lastUpdates)
        pprint.pprint(gameEffectDict)
        pprint.pprint(p1Dict)
        pprint.pprint(p2Dict)
        return gameEffectDict, p1Dict, p2Dict
    def getCanonicalName(name):
        #if '-' in name and name not in Databases.namesWithDash:
            #return name.split('-')[0]
        return name
    def getCanonicalNames(playerDict):
        names = list(playerDict['names'].keys())
        #for i in range(len(names)):
            #if '-' in names[i] and names[i] not in Databases.namesWithDash:
                #names[i] = ScrapingFunctions.getCanonicalName(names[i])
        return names